In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import torch
import pickle
import torch.nn as nn
from persist import PERSIST, HurdleLoss, ExpressionDataset

In [2]:
df_raw = sc.read_10x_h5("/media/chang/HDD-9/chang/thal_count.h5")
df_cpm = sc.read_10x_h5("/media/chang/HDD-9/chang/thal_cpm.h5")

/home/chang/miniconda3/envs/venv3.8/lib/python3.8/site-packages/scanpy/readwrite.py:281: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = AnnData(
/home/chang/miniconda3/envs/venv3.8/lib/python3.8/site-packages/scanpy/readwrite.py:281: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = AnnData(


In [3]:
df_raw = pd.DataFrame.sparse.from_spmatrix(df_raw.X.T,columns=df_raw.obs.index ,index=df_raw.var.index).T
df_raw

,FAM138A,OR4F5,OR4F29,OR4F16,FAM87B,FAM41C,SAMD11,NOC2L,KLHL17,PLEKHN1,...,TTTY4B,BPY2B,DAZ3,DAZ4,BPY2C,TTTY4C,TTTY17C,CDY1,TTTY3,MAFIP
AAACCTGAGAAGATTC_4_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGAATGTGT_4_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGAGCTTCT_4_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGAGTCGGT_4_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGCCGGTAA_4_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCATCAATACCG_16_7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGTCATCAGAAATG_16_7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGTCATCCTAGGGC_16_7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGTCATCGAATGCT_16_7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df_cpm = pd.DataFrame.sparse.from_spmatrix(df_cpm.X.T,columns=df_cpm.obs.index ,index=df_cpm.var.index).T
df_cpm

,FAM138A,OR4F5,OR4F29,OR4F16,FAM87B,FAM41C,SAMD11,NOC2L,KLHL17,PLEKHN1,...,TTTY4B,BPY2B,DAZ3,DAZ4,BPY2C,TTTY4C,TTTY17C,CDY1,TTTY3,MAFIP
AAACCTGAGAAGATTC_4_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGAATGTGT_4_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGAGCTTCT_4_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGAGTCGGT_4_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1114.827148,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGCCGGTAA_4_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCATCAATACCG_16_7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGTCATCAGAAATG_16_7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGTCATCCTAGGGC_16_7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGTCATCGAATGCT_16_7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
with open("/media/chang/HDD-9/chang/thal_annot.txt") as f:
    lines = [line.rstrip() for line in f]
lines = pd.Categorical(lines)
lines

['15', '6', '9', '7', '0', ..., '21', '1', '4', '3', '1']
Length: 135829
Categories (45, object): ['0', '1', '10', '11', ..., '6', '7', '8', '9']

In [6]:
# Arrange raw and CPM data
raw = df_raw.values
cpm = df_cpm.values
gene_names = np.array(df_raw.columns)

# Generate logarithmized and binarized data
binary = (raw > 0).astype(np.float32)
log = np.log(1 + raw)
logcpm = np.log(1 + cpm)

In [7]:
# For data splitting
n = len(raw)
n_train = int(0.8 * n)
n_test = int(0.1 * n)
all_rows = np.arange(n)
np.random.seed(0)
np.random.shuffle(all_rows)
train_inds = all_rows[:n_train]
val_inds = all_rows[n_train:-n_test]
test_inds = all_rows[-n_test:]
print(f'{n} total examples, {len(train_inds)} training examples, {len(val_inds)} validation examples, {len(test_inds)} test examples')

135829 total examples, 108663 training examples, 13584 validation examples, 13582 test examples


In [ ]:
# Jump

In [8]:
# Set up datasets
train_dataset = ExpressionDataset(binary[train_inds], logcpm[train_inds])
val_dataset = ExpressionDataset(binary[val_inds], logcpm[val_inds])

In [9]:
# Number of genes to select
num_genes = (110, 270, 470)
persist_results = {}
device = torch.device('cuda', 0)


In [ ]:
# Set up selector
selector = PERSIST(train_dataset,
                   val_dataset,
                   loss_fn=HurdleLoss(),
                   device=device,
                   hidden=[128, 128])

# Eliminate many candidates
candidates, model = selector.eliminate(target=500, mbsize=2048, max_nepochs=500)

for num in num_genes:
    # Select specific number of genes
    inds, model = selector.select(num_genes=num, mbsize=2048, max_nepochs=500)
    persist_results[num] = df_raw.columns[inds].tolist()
    

using HurdleLoss, starting with lam = 0.01


Training epochs:   0%|          | 0/500 [00:00<?, ?it/s]

lam = 0.010000 yielded 13307 genes
Warm starting model for next iteration
next attempt is lam = 0.100000


Training epochs:   0%|          | 0/500 [00:00<?, ?it/s]

lam = 0.100000 yielded 1416 genes
Warm starting model for next iteration
next attempt is lam = 0.289228


Training epochs:   0%|          | 0/500 [00:00<?, ?it/s]

lam = 0.289228 yielded 757 genes
Warm starting model for next iteration
next attempt is lam = 0.452726


Training epochs:   0%|          | 0/500 [00:00<?, ?it/s]

lam = 0.452726 yielded 593 genes
done, lam = 0.452726 yielded 593 genes


Training epochs:   0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
# Save results
with open('/media/chang/HDD-9/chang/thal.pkl', 'wb') as f:
    pickle.dump(persist_results, f)

In [11]:
# Make cell types numerical
cell_types = pd.Categorical(lines).codes

# Set up datasets
train_dataset = ExpressionDataset(binary[train_inds], cell_types[train_inds])
val_dataset = ExpressionDataset(binary[val_inds], cell_types[val_inds])

persist_class_results = {}
persist_class_idx = {}

In [ ]:
device = torch.device('cuda', 0)

selector = PERSIST(train_dataset,
                   val_dataset,
                   loss_fn=torch.nn.CrossEntropyLoss(),
                   device=device,
                   hidden=[128, 128])

# Eliminate many candidates
candidates, model = selector.eliminate(target=500, mbsize=128, max_nepochs=600)

for num in num_genes:
    # Select specific number of genes
    inds, model = selector.select(num_genes=num, mbsize=128, max_nepochs=600)
    persist_class_results[num] = df_raw.columns[inds].tolist()
    persist_class_idx[num] = inds

In [ ]:
persist_class_results

In [ ]:
# Save results
with open('/media/chang/HDD-9/chang/thal_class.pkl', 'wb') as f:
    pickle.dump(persist_class_results, f)

In [ ]:
#persist_class_results
with open("/media/chang/HDD-9/chang/110_genes.txt", 'w+') as f:
    for line in persist_class_results[110]:
        f.write(f"{line}\n")
with open("/media/chang/HDD-9/chang/270_genes.txt", 'w+') as f:
    for line in persist_class_results[270]:
        f.write(f"{line}\n")
with open("/media/chang/HDD-9/chang/470_genes.txt", 'w+') as f:
    for line in persist_class_results[470]:
        f.write(f"{line}\n")

In [43]:
import torch
import pickle
import matplotlib.pyplot as plt
from persist import models, Accuracy, ExpressionDataset
from torch.utils.data import DataLoader

In [44]:
# Set up datasets
train_dataset = ExpressionDataset(binary[train_inds], cell_types[train_inds])
val_dataset = ExpressionDataset(binary[val_inds], cell_types[val_inds])
test_dataset = ExpressionDataset(binary[test_inds], cell_types[test_inds])


In [ ]:
for num in num_genes:
    # Get inds
    inds = methods[name][num]
        
    # Set up datasets
    train_dataset.set_inds(inds)
    val_dataset.set_inds(inds)
    test_dataset.set_inds(inds)
        
        # Train model
    model = models.MLP(
        input_size=num,
        output_size=train_dataset.output_size,
        hidden=[128, 128]).to(device)
    model.fit(
        train_dataset,
        val_dataset,
        mbsize=512,
        max_nepochs=500,
        loss_fn=torch.nn.CrossEntropyLoss(),
        verbose=False)

        # Validation performance
    test_acc = model.validate(DataLoader(test_dataset, batch_size=1024), Accuracy()).item()
    results[name][num] = test_acc